In [ ]:
import pyPyrTools as ppt
import sys
import numpy as np
sys.path.append('..')
import sfp
%matplotlib inline
import seaborn as sns
from pyPyrTools import JBhelpers as jbh

In [ ]:
ims = []
freq=30
phase = 0
ampl=1
ims.append(sfp.stimuli.log_polar_grating(512,4, freq,phi=phase,ampl=ampl,))
ims.append(sfp.stimuli.log_polar_grating(512,4, w_a=freq,phi=phase,ampl=ampl))
ims.append(sfp.stimuli.log_polar_grating(512,4, freq,freq,phi=phase,ampl=ampl))
ims.append(sfp.stimuli.log_polar_grating(512,4,-freq,freq,phi=phase,ampl=ampl))
ims.append(sfp.stimuli.log_polar_grating(512,4,freq/2,freq,phi=phase,ampl=ampl))
ims.append(sfp.stimuli.log_polar_grating(512,4,freq,freq/2,phi=phase,ampl=ampl))
ims.append(ims[0]+ims[1])

In [ ]:
# Example stimuli
jbh.showIm(ims, ncols=min(len(ims), 4))